In [1]:
import pandas as pd
import requests
from sodapy import Socrata

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [18]:
client = Socrata("data.cityofchicago.org", None)

results = client.get("85ca-t3if", limit=2000, where = "crash_date > '2024-01-19T02:02:00.000'")

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

results_df

,crash_record_id,crash_date_est_i,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,...,latitude,longitude,location,hit_and_run_i,intersection_related_i,statements_taken_i,photos_taken_i,private_property_i,work_zone_i,work_zone_type
0,9f144764070a1bb410525a5936fff3b236692075282b7f...,N,2024-01-23T02:02:00.000,30,NO CONTROLS,UNKNOWN,RAIN,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,...,41.884210486,-87.753839617,"{'type': 'Point', 'coordinates': [-87.75383961...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a1672c7b0a3cf2a6576601f33bf071e74e02698213f5b1...,NaN,2024-01-23T00:53:00.000,15,NO CONTROLS,NO CONTROLS,FREEZING RAIN/DRIZZLE,"DARKNESS, LIGHTED ROAD",PEDESTRIAN,NOT DIVIDED,...,41.731030759,-87.550036403,"{'type': 'Point', 'coordinates': [-87.55003640...",Y,NaN,NaN,NaN,NaN,NaN,NaN
2,a4ff3d4e4faea495dcd464f033d4773b6b6661f90ec946...,NaN,2024-01-23T00:45:00.000,30,NO CONTROLS,NO CONTROLS,FREEZING RAIN/DRIZZLE,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,...,41.942443192,-87.83034231,"{'type': 'Point', 'coordinates': [-87.83034231...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ce5c009cb2ae404d9b019da5ad5976ff6638f3c6fcd37...,Y,2024-01-23T00:30:00.000,20,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,FREEZING RAIN/DRIZZLE,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,...,41.934083674,-87.692642006,"{'type': 'Point', 'coordinates': [-87.69264200...",Y,NaN,NaN,NaN,NaN,NaN,NaN
4,16c83513931d99d14c95f153b1879d5a5303a858fa62e4...,NaN,2024-01-23T00:25:00.000,30,NO CONTROLS,NO CONTROLS,RAIN,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,DIVIDED - W/MEDIAN BARRIER,...,41.96194454,-87.747430051,"{'type': 'Point', 'coordinates': [-87.74743005...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,71022bac45c1a3fc5cd64085cc756702a452f0b3f0a094...,NaN,2024-01-19T02:59:00.000,30,NO CONTROLS,NO CONTROLS,SNOW,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,...,41.750891047,-87.625968505,"{'type': 'Point', 'coordinates': [-87.62596850...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,ef9fabed12502f297965cfd19d709d7387791823b5d591...,NaN,2024-01-19T02:54:00.000,35,NO CONTROLS,NO CONTROLS,SNOW,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,...,41.811965743,-87.703979578,"{'type': 'Point', 'coordinates': [-87.70397957...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,40776f92ba4b0603ebef317afe0fc5057faafda4fb9708...,NaN,2024-01-19T02:20:00.000,20,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,SNOW,UNKNOWN,REAR END,DIVIDED - W/MEDIAN (NOT RAISED),...,42.002504508,-87.660632771,"{'type': 'Point', 'coordinates': [-87.66063277...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,e377fd44fc73483c72ca58c00e5db16a11d799e32e98c9...,NaN,2024-01-19T02:20:00.000,30,NO CONTROLS,NO CONTROLS,SNOW,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,...,41.927925884,-87.727017041,"{'type': 'Point', 'coordinates': [-87.72701704...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
